# Lily.Ai Technical Assessment

The goal of this project is to provide a demonstration of technical skills for the position of Staff Data Scientist at Lily.AI. This notebook is part of a broader set of assets that can be found in the GitHub repository at https://github.com/CYINT/lily-test 

Instructions for the exercise, including a data dictionary for the raw data file, can be found in the accompanying Instructions.pdf file.

The questions we are seeking to answer today are the following:

1. What is Search to Product-View conversion rate?
2. What is Product-View to Order-Complete conversion rate?
3. Are there any interesting patterns from the analysis? 

## Problem approach.

We will divide the problem into four steps.  
1. Load the data into pandas and conduct data exploration.  
2. Calculate the Search to Product-View conversion rate.  
3. Calculate the Product-View to Order-Complete conversion rate.  
4. Form hypotheses about patterns in the data and test those hypotheses using appropriate data science techniques.  

## Step 1. Load the data and conduct an exploratory analysis.

The data available is located in the `event_logs_v2.csv.gz` file. As this file is compressed using gzip, we will import the appropriate packages to unzip the file and load it into a pandas dataframe for exploratory analysis.

As per the instructions, the following `event_code` values are mapped to the description:  
1 - Search  
2 - Product-View  
3 - Order-Complete  

In [1]:
import gzip
import pandas as pd

#Define some constants for improving code readability later
SEARCH = 1
PRODUCT_VIEW = 2
ORDER_COMPLETE = 3

with gzip.open('./event_logs_v2.csv.gz', 'rb') as f:
    data = pd.read_csv(f)

data.head()

,event_code,event_timestamp,user_id,product_id
0,2,2020-11-29T20:04:54Z,13fe7bed08c78ca6f563430b4fd284a1,cc5d78fc86ca1ea6dab7595d8b09a030
1,2,2020-11-29T20:04:59Z,f081f5ce4e8bdd61d618bd64f0647e46,7b82f5b46759a9f238982f2a7c367ba1
2,2,2020-11-29T19:58:19Z,d43f081630bd3a13518ea950213d7bcf,bf91bb4cd7e62b353232e709a30f52be
3,2,2020-11-29T19:58:34Z,5257b92203fd1122a1d9d09e0a236eb2,a1317a5b5531f4dd2d1bbbf9f0ebecca
4,2,2020-11-29T19:58:44Z,0434613b9d24b9f475bde53472787825,0b3ffd66553d9a2440b1ff8500b6b686


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265614 entries, 0 to 1265613
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   event_code       1265614 non-null  int64 
 1   event_timestamp  1265614 non-null  object
 2   user_id          1265614 non-null  object
 3   product_id       1152471 non-null  object
dtypes: int64(1), object(3)
memory usage: 38.6+ MB


The data has been successfully loaded, based on the above output. We can see that there are 1,265,614 records. Let's do some data exploration to see if there is anything unusual that needs correcting in our dataset.

Lets look at the event codes to ensure they are within the range of 1-3, and also convert the timestamp string into a datetime format in UTC timestamp

In [3]:
print('Event code range: ' + str(data['event_code'].min()) + '-' + str(data['event_code'].max()))

data_cleaned = data.copy()
data_cleaned['event_timestamp'] = pd.to_datetime(data['event_timestamp'], utc=True)

print('Event timestamp range: ' + str(data_cleaned['event_timestamp'].min()) + ' - ' + str(data_cleaned['event_timestamp'].max()))
data_cleaned.head()

Event code range: 1-3
Event timestamp range: 2020-11-10 00:00:01+00:00 - 2020-11-29 23:59:59+00:00


,event_code,event_timestamp,user_id,product_id
0,2,2020-11-29 20:04:54+00:00,13fe7bed08c78ca6f563430b4fd284a1,cc5d78fc86ca1ea6dab7595d8b09a030
1,2,2020-11-29 20:04:59+00:00,f081f5ce4e8bdd61d618bd64f0647e46,7b82f5b46759a9f238982f2a7c367ba1
2,2,2020-11-29 19:58:19+00:00,d43f081630bd3a13518ea950213d7bcf,bf91bb4cd7e62b353232e709a30f52be
3,2,2020-11-29 19:58:34+00:00,5257b92203fd1122a1d9d09e0a236eb2,a1317a5b5531f4dd2d1bbbf9f0ebecca
4,2,2020-11-29 19:58:44+00:00,0434613b9d24b9f475bde53472787825,0b3ffd66553d9a2440b1ff8500b6b686


Nothing strange in those two columns. Let's inspect the fact that there are null values in the product_id column. These values should only be from searches as we would not have identified a product at this stage of the funnel. We can test this assumption by looking at the range of `event_code` for product_ids that are null.

In [4]:
null_product_ids = data_cleaned[data_cleaned['product_id'].isnull()]
null_product_ids['event_code'].value_counts()

1    113143
Name: event_code, dtype: int64

It looks like all of the events with null product ids are searches, so we do not have enough evidence of any anomalous data and we should preserve these in the data set.

Let's explore some more:

In [5]:
data_cleaned['event_code'].value_counts()

2    1108507
1     113143
3      43964
Name: event_code, dtype: int64

## Step 2. Calculating Search Conversions

There is a significant amount of data missing from the dataset that would normally be used to track conversions. In this case, we have no way of tying user actions to sessions except temporally. We also cannot tell if a user has taken any other action in between, such as revisiting the home page or changing categories from the menu.  

We will define a conversion as a Product-View that is immediately preceded by a Search from the same user when sorting the data frame by time. In the real world, we might consider a certain threshold of time before a product-view is made, such as over a single day, but for this exercise it should suffice.

In [6]:
data_cleaned_usercats = data_cleaned.copy()
data_cleaned_usercats['user_id'] = data_cleaned['user_id'].astype('category').cat.codes
sorted_data_conversions = data_cleaned_usercats[data_cleaned['event_code'].isin([SEARCH, PRODUCT_VIEW])].sort_values(['user_id', 'event_timestamp']).drop(['event_timestamp'], axis='columns')

sorted_data_conversions['user_diff'] = sorted_data_conversions['user_id'].diff()
sorted_data_conversions['event_diff'] = sorted_data_conversions['event_code'].diff()

sorted_data_conversions['search_conversion'] = sorted_data_conversions.apply(lambda x: 1 if x['user_diff'] == 0 and x['event_diff'] == 1 else 0, axis=1)
search_conversions = sorted_data_conversions['search_conversion'].sum()
viewed_products_from_search = sorted_data_conversions.loc[sorted_data_conversions['search_conversion'] == 1]['product_id']
search_conversions

57713

In [7]:
total_searches = data_cleaned[data_cleaned['event_code'] == SEARCH]['event_code'].count()
search_conversion_rate = search_conversions / total_searches
search_conversion_rate

0.5100890024128757

## 3. Product-View to Order-View conversions

This is straightforward, as we will take the total number of Order-Complete and divide it by the total number of Product-Views. It doesn't matter if search preceded it. We won't filter out a Product-View if a user refreshed the page or if the user was buying multiple items in a single trip -- these might be considerations for a live site where we would track "Add to cart" data.

In [8]:
order_view_conversions = data_cleaned[data_cleaned['event_code'] == ORDER_COMPLETE]['event_code'].count() / data_cleaned[data_cleaned['event_code'] == PRODUCT_VIEW]['event_code'].count()
order_view_conversions

0.039660552436746

## 4. Hypothesis Testing - Are products found through search more likely to convert than those viewed outside of search?

The null hypothesis is that conversion rates for products found through search does not differ from those of products viewed outside of search.  
The alternate hypothesis is that conversion rates for products found through search differ from those of products viewed outside of search.

We will setup the experiment by creating two groups.  
Group A: Products found through search.
Group B: Products found outside of search.

We will conduct A/A and B/B tests on each group to look for evidence of confounding variables. If none are found, we will conduct A/B testing across both groups to determine if a statistically significant difference exists at 95% confidence.


In [9]:
group_a_views = data_cleaned[(data_cleaned['event_code'] == PRODUCT_VIEW) & (data_cleaned['product_id'].isin(viewed_products_from_search))]
print(group_a_views.info())

group_a_orders = data_cleaned[(data_cleaned['event_code'] == ORDER_COMPLETE) & (data_cleaned['product_id']).isin(group_a_views['product_id'])]
group_a_orders.info()

group_a_conversions = group_a_orders['event_code'].count() / group_a_views['event_code'].count()
group_a_conversions

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113079 entries, 2 to 1265601
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   event_code       113079 non-null  int64              
 1   event_timestamp  113079 non-null  datetime64[ns, UTC]
 2   user_id          113079 non-null  object             
 3   product_id       113079 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 4.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1082 entries, 4023 to 1263531
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   event_code       1082 non-null   int64              
 1   event_timestamp  1082 non-null   datetime64[ns, UTC]
 2   user_id          1082 non-null   object             
 3   product_id       1082 non-null   object          

0.009568531734451135

In [10]:
group_b_views = data_cleaned[(data_cleaned['event_code'] == PRODUCT_VIEW) & (~data_cleaned['product_id'].isin(viewed_products_from_search))]
print(group_b_views.info())

group_b_orders = data_cleaned[(data_cleaned['event_code'] == ORDER_COMPLETE) & (data_cleaned['product_id']).isin(group_b_views['product_id'])]
group_b_orders.info()

group_b_conversions = group_a_orders['event_code'].count() / group_b_views['event_code'].count()
group_b_conversions

<class 'pandas.core.frame.DataFrame'>
Int64Index: 995428 entries, 0 to 1265613
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   event_code       995428 non-null  int64              
 1   event_timestamp  995428 non-null  datetime64[ns, UTC]
 2   user_id          995428 non-null  object             
 3   product_id       995428 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 38.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13710 entries, 10 to 1265372
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   event_code       13710 non-null  int64              
 1   event_timestamp  13710 non-null  datetime64[ns, UTC]
 2   user_id          13710 non-null  object             
 3   product_id       13710 non-null  object          

0.0010869696251260765

We have the two groups and the conversion rates. Now we need to do A/A and B/B testing. Let's also import statsmodels to help us.

In [11]:
!pip install statsmodels
from statsmodels.stats.proportion import proportions_ztest

You should consider upgrading via the 'd:\users\dfredriksen\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [12]:
group_a1_views = group_a_views.sample(frac=0.5)
group_a2_views = group_a_views[~group_a_views.index.isin(group_a1_views.index)]
group_a1_orders = group_a_orders[group_a_orders['product_id'].isin(group_a1_views['product_id'])]
group_a2_orders =  group_a_orders[group_a_orders['product_id'].isin(group_a2_views['product_id'])]

group_a1_views_total = group_a1_views['product_id'].count()
group_a1_orders_total = group_a1_orders['product_id'].count()
group_a2_views_total = group_a2_views['product_id'].count()
group_a2_orders_total = group_a2_orders['product_id'].count()

stat, pval = proportions_ztest([group_a1_orders_total, group_a2_orders_total], [group_a1_views_total, group_a2_views_total])
pval

0.7747460605882494

Our p-value is greater than our alpha (0.05) so we do not see any evidence of confounding variables in our A group.

We repeat for the B group

In [13]:
group_b1_views = group_b_views.sample(frac=0.5)
group_b2_views = group_b_views[~group_b_views.index.isin(group_b1_views.index)]
group_b1_orders = group_b_orders[group_b_orders['product_id'].isin(group_b1_views['product_id'])]
group_b2_orders =  group_b_orders[group_b_orders['product_id'].isin(group_b2_views['product_id'])]

group_b1_views_total = group_b1_views['product_id'].count()
group_b1_orders_total = group_b1_orders['product_id'].count()
group_b2_views_total = group_b2_views['product_id'].count()
group_b2_orders_total = group_b2_orders['product_id'].count()

stat, pval = proportions_ztest([group_b1_orders_total, group_b2_orders_total], [group_b1_views_total, group_b2_views_total])
pval

0.32769725073191536

Our p-value is greater than our alpha (0.05) so we do not see any evidence of confounding variables in our B group.

Now we can compare A and B to determine if finding a product through results have an impact on conversion rates.

In [14]:
group_a_orders_total = group_a_orders['product_id'].count()
group_b_orders_total = group_b_orders['product_id'].count()
group_a_views_total = group_a_views['product_id'].count()
group_b_views_total = group_b_views['product_id'].count()

stat, pval = proportions_ztest([group_a_orders_total, group_b_orders_total], [group_a_views_total, group_b_views_total])
pval

1.6834628719039872e-31

Our P-value is less than our alpha of 0.05, which means that we can reject the null hypothesis with 95% confidence and assume that the alternate hypothesis is likely true -- finding a product through search has a different conversion rate than simply viewing a product without searching for it directly.

We can also demonstrate the effect of the difference using Cohen's h formula

$ h = \phi_a - \phi_b $  
$ \phi_i = 2 arcsine(\sqrt{P_i}) $

In [15]:
import numpy as np
phi_a = 2*np.arcsin(np.sqrt(group_a_conversions))
phi_b = 2*np.arcsin(np.sqrt(group_b_conversions))
h = phi_a - phi_b
h

0.13000069343560056

When Cohen's h is 0.1-0.3, the effect size is considered small. As a result, we can conclude that when users find their product through search first, there is a small increase in the likelihood of converting.